TO-DOs:

INCIDENT FILE
- COMPLETED: Parse-out the time from the date/time columns  (Converted to datetime)
- COMPLETED: Calculate the arrival times
- COMPLETED: Create FINAL dataframe for our assignments

STATION FILE
- COMPLETED: Only contain CLV stations


ASSIGNMENTS:
- IN PROGRESS: Google map the CLV station on a map?  (Margot)
- IN PROGRESS: Add clicable names to the pins?  (Margot)
- IN PROGRESS: Pie chart for incident type (Saeger)
- IN PROGRESS: Bar chart by incident for each station (Saeger)
- IN PROGRESS: Bar chart by response time for each station (Saeger)
- ASSIGNED: Hypothesis Testing (Anji)
        - Boxplots?
        - Determine Mean vs Median based on number of outliers?
- COMPLETED: Trends - number of incidents by month compared to previous years (George)
- COMPLETED: Trends - Avg response time by month compared to previous years (George)
- TBD: Pick the highest performing station (lowest avg response times) vs least performing station (highest avg response times)  and look into why  (Further charting for each?)
- TBD: Quality control & code clean-up


%matplotlib inline

In [2]:
# DEPENDENCIES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
import json

# Declare variables for each CSV file
incident_file_df1114 = "data/Fire_Department_Incident_Count - 2011-2014.csv"
incident_file_df1516 = "data/Fire_Department_Incident_Count - 2015-2016.csv"
incident_file_df1718 = "data/Fire_Department_Incident_Count - 2017-2018.csv"
fire_station_data =  "data/Clark_County_Fire_Stations.csv"

#import gmaps
import gmaps

# Import API key
from configure import g_key

ModuleNotFoundError: No module named 'gmaps'

PROCESS INCIDENT DATA HERE

In [3]:
# Step 1: bring-in 2011-2014 data into a df

incident_file_df1718 = pd.read_csv(incident_file_df1718)
print(f"# of rows in 2017-2018 DF: {len(incident_file_df1718)}")

incident_file_df1516 = pd.read_csv(incident_file_df1516)
print(f"# of rows in 2014-2015 DF: {len(incident_file_df1516)}")

incident_file_df1114 = pd.read_csv(incident_file_df1114)
print(f"# of rows in 2011-2014 DF: {len(incident_file_df1114)}")

C:\Users\17737\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# of rows in 2017-2018 DF: 204040
# of rows in 2014-2015 DF: 207549
# of rows in 2011-2014 DF: 317543


In [4]:
# Concatonate previous DFs into a single dataframe

incident_file_df = pd.concat([incident_file_df1114, incident_file_df1516, incident_file_df1718])
incident_file_df = incident_file_df.dropna(how='any')
print(f"# of rows in complete file: {len(incident_file_df)}")

# of rows in complete file: 695296


In [5]:
# New 'lean' DF with only the required columns and ignore the others

incident_file_lean_df = incident_file_df[["Station","Response_Date","Event_Type","First_Unit_Assigned","First_Unit_Arrived","Location","Location_1"]]
incident_file_lean_df.head()

,Station,Response_Date,Event_Type,First_Unit_Assigned,First_Unit_Arrived,Location,Location_1
0,Station 4,2011/07/01 00:00:00+00,Medical,2011/07/01 00:01:00+00,7/1/2011 0:06,S Maryland Pky & E Carson Ave,"(36.16462708, -115.1334076)"
1,Station 44,2011/07/01 00:05:00+00,Medical,2011/07/01 00:06:00+00,7/1/2011 0:12,100 block of Lotus Blossom Ct,"(36.17433167, -115.2525711)"
2,Station 3,2011/07/01 00:22:00+00,Fire,2011/07/01 00:24:00+00,7/1/2011 0:27,N RANCHO DR & W WASHINGTON AVE,"(36.18107986, -115.1791687)"
3,Station 10,2011/07/01 00:22:00+00,Medical,2011/07/01 00:23:00+00,7/1/2011 0:29,2200 block of S Las Vegas Blvd,"(36.1462059, -115.1557083)"
4,Other,2011/07/01 00:23:00+00,Public Need,2011/07/01 00:24:00+00,7/1/2011 0:31,7300 block of Breezy Night Ct,"(36.2447052, -115.2542267)"


In [6]:
# Create new df for rows that DO NOT contain 'none' in the First_Unit_Arrived column

print(f"Number of rows BEFORE clean-up: {len(incident_file_lean_df)}")
incident_file_clean_df = incident_file_lean_df[incident_file_lean_df["First_Unit_Arrived"] != "None"]
print(f"Number of rows AFTER clean-up: {len(incident_file_clean_df)}")
incident_file_clean_df = incident_file_clean_df.drop_duplicates()
print(f"Number of rows AFTER drop_duplicates: {len(incident_file_clean_df)}")
incident_file_clean_df.head()

Number of rows BEFORE clean-up: 695296
Number of rows AFTER clean-up: 648952
Number of rows AFTER drop_duplicates: 642947


,Station,Response_Date,Event_Type,First_Unit_Assigned,First_Unit_Arrived,Location,Location_1
0,Station 4,2011/07/01 00:00:00+00,Medical,2011/07/01 00:01:00+00,7/1/2011 0:06,S Maryland Pky & E Carson Ave,"(36.16462708, -115.1334076)"
1,Station 44,2011/07/01 00:05:00+00,Medical,2011/07/01 00:06:00+00,7/1/2011 0:12,100 block of Lotus Blossom Ct,"(36.17433167, -115.2525711)"
2,Station 3,2011/07/01 00:22:00+00,Fire,2011/07/01 00:24:00+00,7/1/2011 0:27,N RANCHO DR & W WASHINGTON AVE,"(36.18107986, -115.1791687)"
3,Station 10,2011/07/01 00:22:00+00,Medical,2011/07/01 00:23:00+00,7/1/2011 0:29,2200 block of S Las Vegas Blvd,"(36.1462059, -115.1557083)"
4,Other,2011/07/01 00:23:00+00,Public Need,2011/07/01 00:24:00+00,7/1/2011 0:31,7300 block of Breezy Night Ct,"(36.2447052, -115.2542267)"


In [7]:
# Convert Assigned & Arrived times to DateTime, remove timezones
incident_file_clean_df["First_Unit_Assigned"] = pd.to_datetime(incident_file_clean_df["First_Unit_Assigned"])
incident_file_clean_df["First_Unit_Arrived"] = pd.to_datetime(incident_file_clean_df["First_Unit_Arrived"])
incident_file_clean_df["First_Unit_Assigned"] = incident_file_clean_df["First_Unit_Assigned"].dt.tz_localize(None)
incident_file_clean_df.head()

,Station,Response_Date,Event_Type,First_Unit_Assigned,First_Unit_Arrived,Location,Location_1
0,Station 4,2011/07/01 00:00:00+00,Medical,2011-07-01 00:01:00,2011-07-01 00:06:00,S Maryland Pky & E Carson Ave,"(36.16462708, -115.1334076)"
1,Station 44,2011/07/01 00:05:00+00,Medical,2011-07-01 00:06:00,2011-07-01 00:12:00,100 block of Lotus Blossom Ct,"(36.17433167, -115.2525711)"
2,Station 3,2011/07/01 00:22:00+00,Fire,2011-07-01 00:24:00,2011-07-01 00:27:00,N RANCHO DR & W WASHINGTON AVE,"(36.18107986, -115.1791687)"
3,Station 10,2011/07/01 00:22:00+00,Medical,2011-07-01 00:23:00,2011-07-01 00:29:00,2200 block of S Las Vegas Blvd,"(36.1462059, -115.1557083)"
4,Other,2011/07/01 00:23:00+00,Public Need,2011-07-01 00:24:00,2011-07-01 00:31:00,7300 block of Breezy Night Ct,"(36.2447052, -115.2542267)"


In [8]:
# Calculate Response Time column
incident_file_clean_df["Response Time"] = (incident_file_clean_df["First_Unit_Arrived"] - 
                                           incident_file_clean_df["First_Unit_Assigned"]).astype('timedelta64[m]')
incident_file_clean_df.head()

,Station,Response_Date,Event_Type,First_Unit_Assigned,First_Unit_Arrived,Location,Location_1,Response Time
0,Station 4,2011/07/01 00:00:00+00,Medical,2011-07-01 00:01:00,2011-07-01 00:06:00,S Maryland Pky & E Carson Ave,"(36.16462708, -115.1334076)",5.0
1,Station 44,2011/07/01 00:05:00+00,Medical,2011-07-01 00:06:00,2011-07-01 00:12:00,100 block of Lotus Blossom Ct,"(36.17433167, -115.2525711)",6.0
2,Station 3,2011/07/01 00:22:00+00,Fire,2011-07-01 00:24:00,2011-07-01 00:27:00,N RANCHO DR & W WASHINGTON AVE,"(36.18107986, -115.1791687)",3.0
3,Station 10,2011/07/01 00:22:00+00,Medical,2011-07-01 00:23:00,2011-07-01 00:29:00,2200 block of S Las Vegas Blvd,"(36.1462059, -115.1557083)",6.0
4,Other,2011/07/01 00:23:00+00,Public Need,2011-07-01 00:24:00,2011-07-01 00:31:00,7300 block of Breezy Night Ct,"(36.2447052, -115.2542267)",7.0


In [9]:
# Perform analysis
#incident_file_clean_df.sort_values("Response Time",ascending=False)

In [ ]:
#Create FINAL, cleaned dataframe, ready for analysis, charts, maps, and hypothesis testing!!!
print(f"Rows BEFORE final clean-up: {len(incident_file_clean_df)}")
incident_file_final_df = incident_file_clean_df[incident_file_clean_df["First_Unit_Arrived"] > 
                                                incident_file_clean_df["First_Unit_Assigned"]]
print(f"Rows AFTER final clean-up: {len(incident_file_final_df)}")
incident_file_final_df.head()

In [ ]:
######################################################################################
##
##  TEMPORARY CODE FOR ANALYSIS - REMOVE
##
######################################################################################

#df = incident_file_final_df
#df2 = df.loc[(df["First_Unit_Assigned"] > "2018-10-31") & (df["First_Unit_Assigned"] < "2018-12-01")]
#df2 = df.sort_values(["Location","First_Unit_Assigned"])
#df2.head(100)
#print(f"Rows BEFORE drop_duplicates: {len(df)}")
#df=df.drop_duplicates()
#print(f"Rows AFTER drop_duplicates: {len(df)}")


######################################################################################
##
## TEMPORARY CODE - REMOVE
##
######################################################################################

In [ ]:
#incident_file_final_df.count()
#incident_file_final_df.sort_values("Response Time",ascending=False)
#incident_file_final_df.max()
#incident_file_final_df.max()
#incident_file_final_df.count()
#incident_file_final_df.describe

PROCESS FIRE STATION DATA (MARGOT'S SECTION)

In [ ]:
# Bring-in fire station data
station_df = pd.read_csv(fire_station_data)
print(f"# of rows in the Station data file: {len(station_df)}")
#station_df.head()

In [ ]:
distinct_stations = incident_file_clean_df["Station"].unique()
clv_stations_df = station_df[station_df["CITY_CODE"] == "CLV"]
clv_stations_df.head()

INCIDENT TYPES & COUNTS (SAEGER)

In [ ]:
incident_file_clean_df["First_Unit_Assigned"]

In [ ]:
incident_file_clean_df.head()

In [ ]:
Stations = incident_file_clean_df["Station"].value_counts()

In [ ]:
Stations.head(20)

In [ ]:
plot_pandas= Stations.plot(kind = "bar", color = 'red')
plt.xlabel('Station')
plt.ylabel('Number of Calls')
plt.title("Amount of Calls by Station")
plt.show()

In [ ]:
#Analyzing incidents by Event Type
Calls = incident_file_clean_df["Event_Type"].value_counts()

In [ ]:
Calls

In [ ]:
Calls.plot(kind='pie', y='Event_Type', startangle = 360, autopct ='%1.1f%%' )
plt.title("Event Types")
plt.show()

In [ ]:
incident_file_clean_df.loc[incident_file_clean_df["Event_Type"] == "Medical"]

RESPONSE TIMES ANALYSIS (GEORGE)

In [ ]:
# New DF with only necessary columns, including Location to help look for duplicate rows
response_df = incident_file_final_df[['First_Unit_Assigned','Response Time']]
#response_df = incident_file_final_df[['Station','First_Unit_Assigned','Response Time','Location']]

# Create Year & Month columns based on First_Unit_Assigned column
response_df['Year'] = pd.DatetimeIndex(response_df['First_Unit_Assigned']).year
response_df['Month'] = pd.DatetimeIndex(response_df['First_Unit_Assigned']).month
#response_df.set_index(['Year','Month'])

response_df.head()

In [ ]:
# Pivot the response_df for AVG RESPONSE TIME and prep for year-over-year line chart
avg_pivoted_df = response_df.pivot_table(index='Month', values ='Response Time',columns='Year',aggfunc='mean')
avg_pivoted_df

In [ ]:
avg = avg_pivoted_df.plot(kind='line', xlabel='Month Number', ylabel='Average Response Time (min)', grid=True,
                          title='Year Over Year Avg Response Time', figsize=(10,8),linewidth=4)
avg.legend(bbox_to_anchor=(1,1.02), loc='upper left')

plt.show()

In [ ]:
# Pivot the response_df for INCIDENT COUNTS and prep for year-over-year line chart
count_pivoted_df = response_df.pivot_table(index='Month', values ='Response Time',columns='Year',aggfunc='count')
count_pivoted_df

In [ ]:
count = count_pivoted_df.plot(kind='line', xlabel='Month Number', ylabel='Number of Incidents', grid=True,
                          title='Year Over Year Number of Incidents', figsize=(10,8),linewidth=4)
count.legend(bbox_to_anchor=(1,1.02), loc='upper left')

plt.show()

In [ ]:
incident_file_clean_df.head()

In [ ]:
incident_file_clean_df.shape

HYPOTHESIS TESTING (ANJI)

In [ ]:

# Bring-in fire station data
station_df = pd.read_csv(fire_station_data)
print(f"# of rows in the Station data file: {len(station_df)}")
#station_df.head()

In [ ]:
distinct_stations = incident_file_clean_df["Station"].unique()
clv_stations_df = station_df[station_df["CITY_CODE"] == "CLV"]
clv_stations_df.head()

In [ ]:

#Access maps with unique API key
gmaps.configure(api_key=g_key)


In [ ]:
#create a map of Firehouse locations
station_locations = clv_stations_df[['LAT','LONG']]

fig = gmaps.figure()
markers = gmaps.marker_layer(station_locations)


   
fig.add_layer(markers)
fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Facility Name</dt><dd>{FACILITY_NAME}</dd>
<dt>Latitude</dt><dd>{LAT}</dd>
<dt>Longitude</dt><dd>{LONG}</dd>
</dl>
"""
# Store the DataFrame Row
# update with your DataFrame name
location_info = [info_box_template.format(**row) for index, row in clv_stations_df.iterrows()]
locations = clv_stations_df[["LAT", "LONG"]]

In [ ]:
# Add marker layer ontop of map
markers = gmaps.marker_layer(locations,info_box_content=location_info)
fig.add_layer(markers)
fig